## Deliverable 2. Create a Customer Travel Destinations Map.

In [33]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [34]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,60.46,67,40,14.97,scattered clouds
1,1,Albany,US,42.6001,-73.9662,21.31,76,10,4.18,clear sky
2,2,Castro,BR,-24.7911,-50.0119,61.56,84,2,2.10,clear sky
3,3,Itarema,BR,-2.9248,-39.9167,78.80,78,2,11.07,clear sky
4,4,Atuona,PF,-9.8000,-139.0333,78.80,77,12,17.09,few clouds


In [35]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [36]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Itarema,BR,-2.9248,-39.9167,78.80,78,2,11.07,clear sky
4,4,Atuona,PF,-9.8000,-139.0333,78.80,77,12,17.09,few clouds
6,6,Vaini,TO,-21.2000,-175.2000,75.36,100,100,4.61,light intensity shower rain
11,11,Kaeo,NZ,-35.1000,173.7833,77.32,96,99,6.44,overcast clouds
12,12,Faanui,PF,-16.4833,-151.7500,81.01,75,100,20.13,light rain
17,17,Phan Thiet,VN,10.9333,108.1000,85.98,55,70,11.07,broken clouds
22,22,Avarua,CK,-21.2078,-159.7750,80.65,78,20,9.22,few clouds
24,24,Alice Springs,AU,-23.7000,133.8833,87.35,23,0,5.75,clear sky
26,26,Carnarvon,AU,-24.8667,113.6333,76.28,48,6,21.09,clear sky
34,34,Lorengau,PG,-2.0226,147.2712,83.64,76,99,7.54,light rain


In [37]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                164
City                   164
Country                164
Lat                    164
Lng                    164
Max Temp               164
Humidity               164
Cloudiness             164
Wind Speed             164
Current Description    164
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [38]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Itarema,BR,78.80,clear sky,-2.9248,-39.9167,
4,Atuona,PF,78.80,few clouds,-9.8000,-139.0333,
6,Vaini,TO,75.36,light intensity shower rain,-21.2000,-175.2000,
11,Kaeo,NZ,77.32,overcast clouds,-35.1000,173.7833,
12,Faanui,PF,81.01,light rain,-16.4833,-151.7500,
...,...,...,...,...,...,...,...
646,Ciudad Bolivar,VE,77.54,scattered clouds,8.1222,-63.5497,
657,Mahibadhoo,MV,81.09,scattered clouds,3.7833,72.9667,
669,Sassandra,CI,77.61,overcast clouds,4.9500,-6.0833,
670,Kieta,PG,86.56,light rain,-6.2167,155.6333,


In [39]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")

        

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [41]:
# 7. Drop the rows where there is no Hotel Name.
drop_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
drop_hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Itarema,BR,78.80,clear sky,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
4,Atuona,PF,78.80,few clouds,-9.8000,-139.0333,Villa Enata
6,Vaini,TO,75.36,light intensity shower rain,-21.2000,-175.2000,Keleti Beach Resort
11,Kaeo,NZ,77.32,overcast clouds,-35.1000,173.7833,Bushido Lodge
12,Faanui,PF,81.01,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
17,Phan Thiet,VN,85.98,broken clouds,10.9333,108.1000,Hotel Saigon-PT
22,Avarua,CK,80.65,few clouds,-21.2078,-159.7750,Paradise Inn
24,Alice Springs,AU,87.35,clear sky,-23.7000,133.8833,Desert Palms Alice Springs
26,Carnarvon,AU,76.28,clear sky,-24.8667,113.6333,Hospitality Carnarvon
34,Lorengau,PG,83.64,light rain,-2.0226,147.2712,Lorengau Harbourside Hotel


In [42]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"


# 8b. Export the City_Data into a csv
drop_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [43]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in drop_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = drop_hotel_df[["Lat", "Lng"]]

In [44]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))